In [ ]:
# Initialize GAPID environment and set up some helper methods
from helpers import gapid_env
from helpers import image_formats
from helpers import jupyter_helpers
from helpers import replay
from helpers import trace as gapid_trace
from helpers import vulkan
%matplotlib widget
from importlib import reload

def reload_helpers():
    reload(gapid_env)
    reload(image_formats)
    reload(jupyter_helpers)
    reload(replay)
    reload(gapid_trace)
    reload(vulkan)


reload_helpers()

import matplotlib
import os

pathdir = globals()['_dh'][0]
env = gapid_env.gapid_env(os.path.normpath(os.path.join(pathdir, "..", "build", "Debug")))
env.ignore_debug_messages = True
env.ignore_layer_info_messages = True


In [ ]:
# Select and load trace file
from ipyfilechooser import FileChooser

reload_helpers()

fc = FileChooser()
if env.current_trace:
    fc.default_path = os.path.dirname(env.current_trace)
    fc.default_filename = os.path.basename(env.current_trace)

fc.filter_pattern = "*.trace"
fc.title = "<b>Select Trace</b>"
trace = None
def change_file(chooser):
    global trace
    current_trace = os.path.join(chooser.selected_path, chooser.selected_filename)
    trace =  gapid_trace.load_trace(env, current_trace)
fc.register_callback(change_file)
display(fc)

In [ ]:
# Report stats about the trace
import json
reload_helpers()

cmd_stats = trace.get_stats()
print(json.dumps(cmd_stats, indent="    "))


In [ ]:
# Count the draw commands actually submitted
from collections import defaultdict

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & gapid_trace.MID_EXECUTION) == 0)]

submitted_draws = []
submitted_dispatches = []
for x in queue_submits:
    executed_cbs = trace.executed_commands[x]
    for y in executed_cbs:
        submitted_draws.extend([x for x in y[1] if "Draw" in trace.commands[x]["name"]])
        submitted_dispatches.extend([x for x in y[1] if "Dispatch" in trace.commands[x]["name"]])

l = defaultdict(list)
for x in submitted_draws:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Draws: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

l = defaultdict(list)
for x in submitted_dispatches:
    l[trace.commands[x]["name"]].append(x)

print(f"Total Dispatches: {len(submitted_draws)}")
for k in l.keys():
    print(f"  {k}: {len(l[k])}")

In [ ]:
# Render trace on the screen
reload_helpers()

create_swapchain = [x for x in trace.commands if x["name"] == "vkCreateSwapchainKHR"][0]

opts = replay.replay_options(env, trace)
#opts.use_callback_swapchain()
opts.add_layer(os.path.abspath("on_screen.cpp"), {
    "start_idx": 1,
    "width": create_swapchain["pCreateInfo"]["imageExtent"]["width"],
    "height": create_swapchain["pCreateInfo"]["imageExtent"]["height"],
}, None)

replay.replay(env, opts)

In [ ]:
# Collect screenshots from the trace (Start with the first 10 images that were properly rendered)
screenshots_to_collect = 10
reload_helpers()

import itertools
import base64
from matplotlib import pyplot

queue_submits = [i for i, v in enumerate(trace.commands) if (v["name"] == "vkQueueSubmit" and (v["tracer_flags"] & gapid_trace.MID_EXECUTION) == 0)]

idx = 0
executed_cbs = trace.executed_commands[queue_submits[idx]]
draws = []
while (len(draws) == 0):
    idx += 1
    executed_cbs = trace.executed_commands[queue_submits[idx]]

    for y in executed_cbs:
        draws.extend([(y[0], x) for x in range(len(y[1])) if "Draw" in trace.commands[y[1][x]]["name"]])

draws = itertools.groupby(draws, lambda x: x[0])
total_draws = 0
config = {"screenshot_locations": [
    {
        "submit_index": queue_submits[idx],
        "command_buffers": []
    }
], "num_images_per_draw": 1}

for x in draws:
    cbs = {
        "command_buffer": x[0],
    }
    draw_calls = list(x[1])
    draws_to_take = screenshots_to_collect - total_draws
    if len(draw_calls) < draws_to_take:
        draws_to_take = len(draw_calls)
    cbs["indices"] = [x[1] for x in draw_calls[:draws_to_take]]
    config["screenshot_locations"][0]["command_buffers"].append(cbs)
opts = replay.replay_options(env, trace)

opts.use_callback_swapchain()

def on_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'[{timestamp}] {level} :: {message}')

opts.set_message_callback(on_message)

def on_data(timestamp, data):
    print(f'{timestamp} -- {data["width"], data["height"], data["format"], len(data["data"])}')
    img = image_formats.ToNumpyArray(data["format"], data["width"], data["height"], base64.b64decode(data["data"]))
    # Todo: Actually display this data!
    pyplot.imshow(img)
    pyplot.show()

def on_layer_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'screenshot.cpp:: [{timestamp}] {level} :: {message}')

opts.add_layer(os.path.abspath("screenshot.cpp"), config, data_callback=on_data, message_callback=on_layer_message)

replay.replay(env, opts)

In [ ]:
# Collect screenshots from something rendering color!
screenshots_to_collect = 1000
reload_helpers()
import itertools
import base64
import re
import sys
import PIL
from matplotlib import pyplot
from IPython.display import display, HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128


# Step 1 find all of the 
queue_submits = trace.find_command_indices("vkQueueSubmit")
renderpass_creates = {trace.commands[x]["pRenderPass"]: x for x in trace.find_command_indices("vkCreateRenderPass", True)}

def get_first_renderpass_with_color():
    for x in queue_submits:
        rp_begins = trace.get_submitted_commands_matching(x, re.compile("vkCmdBeginRenderPass"))
        for c in rp_begins:
            for d in c[1]:
                rp_begin = trace.commands[d[1]]
                render_pass = rp_begin["pRenderPassBegin"]["renderPass"]
                rp_creation = trace.commands[renderpass_creates[render_pass]]
                color_attachments = [x for x in rp_creation["pCreateInfo"]["pSubpasses"][0]["pColorAttachments"] if x["attachment"] != vulkan.VK_ATTACHMENT_UNUSED]
                if (len(color_attachments)):
                    has_draws = trace.get_submitted_commands_matching(x, re.compile(".*Draw.*"))
                    if not has_draws:
                        continue
                    return (x, c[0], d[0])
    return None

cb_with_color = get_first_renderpass_with_color()
if cb_with_color == None:
    print("No renderpasses that actally draw anything (odd)")
    sys.exit(-1)

cbs_matching = [x for x in trace.get_submitted_commands_matching(cb_with_color[0], re.compile(".*Draw.*")) if x[0] == cb_with_color[1]]
draws = [x[0] for x in cbs_matching[0][1] if x[0] > cb_with_color[2]][:screenshots_to_collect]
config = {"screenshot_locations": [
    {
        "submit_index": cb_with_color[0],
        "command_buffers": [{
            "command_buffer": cb_with_color[1],
            "indices": draws
        },]
    }
], "num_images_per_draw": 1}

opts = replay.replay_options(env, trace)
print(config)

opts.use_callback_swapchain()

def on_message(timestamp, level, message):
    if level == "Debug":
        return
    print(f'[{timestamp}] {level} :: {message}')

opts.set_message_callback(on_message)
imgs = []
def on_data(timestamp, data):
    global imgs
    dat = base64.b64decode(data["data"])
    img = image_formats.ToNumpyArray(data["format"], data["width"], data["height"], dat)
    imgs.append(img.astype('uint8'))

def on_layer_message(timestamp, level, message):
    if level == "Debug":
        return
    if level == "Info":
        return
    print(f'screenshot.cpp:: [{timestamp}] {level} :: {message}')

opts.add_layer(os.path.abspath("screenshot.cpp"), config, data_callback=on_data, message_callback=on_layer_message)

replay.replay(env, opts)
fig, anim = jupyter_helpers.plot_sequence_images(imgs[1:])
pyplot.show(fig)


In [ ]:
# Get information about rendering times
reload_helpers()

draw_batches = 1

import matplotlib.pyplot as plt
import numpy
import random
import math
from io import BytesIO
import imageio
import ipywidgets

frames = trace.get_rendering_info()
img = ipywidgets.Image()

current_frame = -1
current_renderpass = -1
current_drawcall = -1

frameSlider = ipywidgets.IntSlider(value = 0, min = 0, continuous_update=False, max=len(frames), step=1, description='Frame: ', layout=ipywidgets.Layout(width='100%'))
renderpassSlider = ipywidgets.IntSlider(value = 0, min = -1, continuous_update=False, max=len(frames[current_frame]), step=1, description='Renderpass: ', layout=ipywidgets.Layout(width='100%'))
drawCallSlider = ipywidgets.IntSlider(value = 0, min = -1, continuous_update=False, max=len(frames[current_frame][current_drawcall].draw_calls), step=1, description='DrawCall: ', layout=ipywidgets.Layout(width='100%'))

imgs = []

last_draw_call_acquire = -2*draw_batches

def do(change):
    global current_frame
    global current_renderpass
    global current_drawcall
    global imgs
    global frames
    global last_draw_call_acquire
    global img
    dirty = False
    if (current_frame == frameSlider.value and
        current_renderpass == renderpassSlider.value and
        current_drawcall == drawCallSlider.value):
        return
    if frameSlider.value != current_frame:
        current_frame = frameSlider.value
        renderpassSlider.value = 0
        renderpassSlider.max = len(frames[current_frame]) -1
        current_renderpass = 0
        current_drawcall = 0
        dirty = True
    if current_renderpass != renderpassSlider.value or dirty:
        dirty = True
        current_renderpass = renderpassSlider.value
        drawCallSlider.value = 0
        if len(frames[current_frame]) > 0:
            drawCallSlider.max = len(frames[current_frame][current_renderpass].draw_calls) - 1
        else:
            drawCallSlider.value = -1
            drawCallSlider.max = -1
            drawCallSlider.value = -1
    current_drawcall = drawCallSlider.value

    if len(frames[current_frame]) <= current_renderpass:
        img.value = bytes()
        return
    if len(frames[current_frame][current_renderpass].draw_calls) <= current_drawcall:
        img.value = bytes()
        return
    if dirty or last_draw_call_acquire <= current_drawcall - draw_batches or current_drawcall < last_draw_call_acquire:
        img.value = bytes()
        last_draw_call_acquire = current_drawcall
        imgs = replay.screenshot_helper(env, trace, frames[current_frame][current_renderpass].draw_calls[last_draw_call_acquire:last_draw_call_acquire+draw_batches], 1, True)
    if current_drawcall >= 0:
        ii = imgs[current_drawcall - last_draw_call_acquire]
        #ii = numpy.resize(ii, (ii.shape[0], ii.shape[1], 3))
        buff = BytesIO()
        imageio.imwrite(buff, ii, format='png')
        buff.seek(0)
        img.value = buff.read()


frameSlider.observe(do)
renderpassSlider.observe(do)
drawCallSlider.observe(do)

display(frameSlider)
display(renderpassSlider)
display(drawCallSlider)
display(img)
do(1)

In [ ]:
# Get information about rendering times
reload_helpers()
draw_batches = 1

import random
import math
from io import BytesIO
import ipywidgets
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [7, 5]
frames = trace.get_rendering_info()

timestamps = replay.timestamp_helper(env, trace, frames[0])
render_passes = []
for submit in timestamps:
    for command_buffer in submit:
        for renderpass in command_buffer['render_passes']:
            render_passes.append(renderpass)

fig = plt.figure()
ax = ax=fig.add_subplot(111)#fig.add_axes([0,0,1,1])

top_render_passes = sorted(sorted(range(len(render_passes)), key=lambda i: render_passes[i]['end_time'] - render_passes[i]['start_time'])[-10:])

bg = ax.bar([f"{x}-" + str(render_passes[x]['render_pass']) for x in top_render_passes],
    [render_passes[x]['end_time'] - render_passes[x]['start_time'] for x in top_render_passes])
ax.bar_label(bg, label_type='center')
plt.show()
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=-15, ha='center')